In [1]:
from pymystem3 import Mystem
from os import listdir

In [2]:
import pymorphy2
import pymorphy2_dicts_ru

In [3]:
# морфологический анализатор PyMorphy
pymorphy = pymorphy2.MorphAnalyzer()

In [4]:
# функция конкатенации списков
def concat(lists):
    result = []
    for lst in lists:
        for elem in lst:
            result.append(elem)
    return result

In [5]:
def get_pos(word):
    params = pymorphy.parse(word)[0]
    return params.tag.POS

In [6]:
# морфологический анализатор Mystem
mystem = Mystem()

In [7]:
# директория хранения текстовых файлов для тестирования
text_path = 'texts'

# считывание текстовых файлов из заданной директории в список текстов
texts = [ open(text_path + '/' + filename, 'r').read() for filename in listdir(text_path) ]

In [8]:
# предикат: является ли слово разделителем
def is_separator(word):
    return word.get('analysis') == None

# получить текст слова
def get_text(word):
    return word['text']

# извлечение из текста множества всех разделителей
separators = set(concat([ list(map(get_text, filter(is_separator, mystem.analyze(text)))) for text in texts ]))

In [9]:
# посмотрим на них
separators

{'\n',
 ' ',
 ' (',
 ' «',
 ' ‒ ',
 ' –',
 ' – ',
 ' — ',
 ' — «',
 '!',
 ') ',
 '), ',
 '); ',
 ',',
 ', ',
 ', ‒ ',
 ', – ',
 ', — ',
 '-',
 '- ',
 '.',
 '. ',
 '/',
 '1917',
 '1941',
 ':\n',
 ': ',
 ': «',
 '; ',
 '?',
 '? ',
 '«',
 '»',
 '» ',
 '» (',
 '» ‒ ',
 '», ',
 '», – ',
 '– ',
 '…',
 '… '}

In [10]:
# дефис
connectSeparators = set(['- ', '-'])

# элементы, ошибочно выбранные разделителями (и пробел)
excludeSeparators = set(['1917', '1941', ' ']) | connectSeparators

# разделители, завершающие простые предложения
endSeparators = separators - excludeSeparators
endSeparators

{'\n',
 ' (',
 ' «',
 ' ‒ ',
 ' –',
 ' – ',
 ' — ',
 ' — «',
 '!',
 ') ',
 '), ',
 '); ',
 ',',
 ', ',
 ', ‒ ',
 ', – ',
 ', — ',
 '.',
 '. ',
 '/',
 ':\n',
 ': ',
 ': «',
 '; ',
 '?',
 '? ',
 '«',
 '»',
 '» ',
 '» (',
 '» ‒ ',
 '», ',
 '», – ',
 '– ',
 '…',
 '… '}

In [11]:
# Получение списка возможных разборов словоформы без маловероятных случаев
def get_parse_list(word):
    result = list(filter(lambda x: x.score > 0.1, pymorphy.parse(word)))
    if result == []:
        return pymorphy.parse(word)
    else:
        return result


# Получение множества падежей для словоформы
# filter_pos - часть речи, по которой фильтруется поиск
# если ее значение None, то ищем все
def case_set(word, filter_pos=None):
    parse_list = get_parse_list(word)
    result = set()
    
    for x in parse_list:
        if filter_pos != None and x.tag.POS != filter_pos:
            continue
        
        case = x.tag.case
        if case != None:
            result.add(case)
    
    return result


# Получение множества частей речи для словоформы
def pos_set(word):
    parse_list = get_parse_list(word)
    result = set()
    
    for x in parse_list:
        pos = x.tag.POS
        if pos != None:
            result.add(pos)
    
    return result


# Есть ли общие падежи для двух словоформ?
def have_common_case(w1, w2, pos1=None, pos2=None):
    if pos2 == 'NOUN' and is_unchangeable(w2):
        return True
    
    return len(case_set(w1, pos1) & case_set(w2, pos2)) > 0


# Является ли слово неизменяемым?
def is_unchangeable(word):
    parse_list = pymorphy.parse(word)
    return any(map(lambda x: 'Fixd' in x.tag, parse_list))

# Является ли отрицательной частицей?
def is_negation(word):
    return word in ['не', 'ни']

In [12]:
# Печать обработанной синтаксически связанной группы
def print_group(group):
    if len(group) > 1:
        print('{', end='')

        for word in group:
            print(word, end='')

        print('}', end='')
    else:
        for word in group:
            print(word, end='')


# Печать обработанного простого предложения
def print_sentence(sentence):
    if isinstance(sentence, list):
        print('[', end='')

        for group in sentence:
            print_group(group)

        print(']', end='')
    else:
        print(sentence, end='')


# Печать обработанного текста        
def print_analyzed_text(text):
    for sentence in text:
        print_sentence(sentence)

In [13]:
for text in texts:
    # извлечение из текста списка слова со всеми характеристиками с помощью морфологического анализатора
    words = list(map(get_text, mystem.analyze(text)))
    
    # склейка слов, написанных через дефис
    connected_words = []
    
    i = 0
    while i < len(words):
        if i + 2 < len(words) and words[i+1] in connectSeparators:
            connected_words.append(words[i] + words[i+1] + words[i+2])
            i += 2
        else:
            connected_words.append(words[i])
        i += 1
    
    words = connected_words
    
    # список всех сегментов-предложений, полученных из обрабатываемого текста
    analyzed_text = []
    # список сегментов в рамках предложения
    sentence = []
    # список слов неразрывной синтаксической группы
    group = []
    
    i = 0
    while i < len(words):
        word = words[i]
        if word in endSeparators:
            if group != []:
                sentence.append(group)
                group = []

            if sentence != []:
                analyzed_text.append(sentence)
                sentence = []

            analyzed_text.append(word)
        elif word in separators and group == []:
            sentence.append([word])
        elif i + 2 < len(words) and words[i+1] not in endSeparators:
            pos1 = pos_set(words[i])
            pos2 = pos_set(words[i+2])
            if len(set(['PRTF', 'ADJF']) & pos1) > 0 and len(set(['PRTF', 'ADJF']) & pos2) > 0 and have_common_case(words[i], words[i+2]) or len(set(['PRTS', 'ADJS', 'PRTF', 'ADJF']) & pos1) > 0 and len(set(['PRTS', 'ADJS', 'PRTF', 'ADJF']) & pos2) > 0:
                if i >= 2 and words[i-1] not in endSeparators and ('PREP' in pos_set(words[i-2]) or is_negation(words[i-2])):
                    sentence.pop()
                    sentence.pop()
                    group = [ words[i-2], words[i-1] ]
                    if i >= 4 and words[i-3] not in endSeparators and ('PREP' in pos_set(words[i-4]) or is_negation(words[i-4])):
                        sentence.pop()
                        sentence.pop()
                        group = [ words[i-4], words[i-3] ] + group
                else:
                    group = []
                group += [ word, words[i+1], words[i+2] ]
                i += 2
                pos1 = pos2
                pos2 = pos_set(words[i+2])
                while i + 2 < len(words) and words[i+1] not in endSeparators:
                    if len(set(['PRTF', 'ADJF']) & pos2) > 0 and have_common_case(words[i], words[i+2]) or len(set(['PRTS', 'ADJS', 'PRTF', 'ADJF']) & pos2) > 0:
                        group += [ words[i+1], words[i+2] ]
                        i += 2
                        pos1 = pos2
                        pos2 = pos_set(words[i+2])
                        continue
                    elif len(set(['NOUN', 'NPRO']) & pos2) > 0 and have_common_case(words[i], words[i+2], 'ADJF', 'NOUN'):
                        group += [ words[i+1], words[i+2] ]
                        i += 2
                        pos1 = pos2
                        pos2 = pos_set(words[i+2])
                    break
                if i + 2 < len(words) and words[i+1] in connectSeparators:
                    group += [ words[i+1], words[i+2] ]
                    i += 2
                sentence.append(group)
                group = []
            elif 'ADJF' in pos1 and len(set(['NOUN', 'NPRO']) & pos2) > 0 and have_common_case(words[i], words[i+2], 'ADJF'):
                if i >= 2 and words[i-1] not in endSeparators and ('PREP' in pos_set(words[i-2]) or is_negation(words[i-2])):
                    sentence.pop()
                    sentence.pop()
                    group = [ words[i-2], words[i-1] ]
                    if i >= 4 and words[i-3] not in endSeparators and ('PREP' in pos_set(words[i-4]) or is_negation(words[i-4])):
                        sentence.pop()
                        sentence.pop()
                        group = [ words[i-4], words[i-3] ] + group
                else:
                    group = []
                group += [ word, words[i+1], words[i+2] ]
                i += 2
                if i + 2 < len(words) and words[i+1] in connectSeparators:
                    group += [ words[i+1], words[i+2] ]
                    i += 2
                sentence.append(group)
                group = []
            elif 'PREP' in pos1 and len(set(['NOUN', 'NPRO']) & pos2) > 0:
                if i >= 2 and words[i-1] not in endSeparators and is_negation(words[i-2]):
                    sentence.pop()
                    sentence.pop()
                    group = [ words[i-2], words[i-1] ]
                else:
                    group = []
                group += [ word, words[i+1], words[i+2] ]
                i += 2
                if i + 2 < len(words) and words[i+1] in connectSeparators:
                    group += [ words[i+1], words[i+2] ]
                    i += 2
                sentence.append(group)
                group = []
            elif len(set(['PRTF', 'PRTS']) & pos1) > 0 and len(set(['NOUN', 'NPRO']) & pos2) > 0:
                if i >= 2 and words[i-1] not in endSeparators and ('PREP' in pos_set(words[i-2]) or is_negation(words[i-2])):
                    sentence.pop()
                    sentence.pop()
                    group = [ words[i-2], words[i-1] ]
                    if i >= 4 and words[i-3] not in endSeparators and ('PREP' in pos_set(words[i-4]) or is_negation(words[i-4])):
                        sentence.pop()
                        sentence.pop()
                        group = [ words[i-4], words[i-3] ] + group
                else:
                    group = []
                group += [ word, words[i+1], words[i+2] ]
                i += 2
                if i + 2 < len(words) and words[i+1] in connectSeparators:
                    group += [ words[i+1], words[i+2] ]
                    i += 2
                sentence.append(group)
                group = []
            else:
                sentence.append([word])
        else:
            sentence.append([word])
        i += 1
    else:
        if group != []:
            sentence.append(group)
        
        if sentence != []:
            analyzed_text.append(sentence)
    
    print_analyzed_text(analyzed_text)

[{Профессорская дача} {на берегу} {Финского залива}]. [{В отсутствие} хозяина], [друга {моего отца}], [{нашей семье} позволялось там жить]. [Даже {спустя десятилетия} помню], [как {после утомительной дороги} {из города} меня обволакивала прохлада {деревянного дома}], [как собирала растрясшееся], [распавшееся {в экипаже} тело]. [{Эта прохлада} не была связана {со свежестью}], [скорее], [как ни странно], — [{с упоительной затхлостью}], [в которой слились ароматы {старых книг} и {многочисленных океанских трофеев}], [непонятно как {доставшихся профессору-юристу}]. [Распространяя {солоноватый запах}], [{на полках} лежали {засушенные морские звёзды}], [{перламутровые раковины}], [{резные маски}], [{пробковый шлем} и даже игла рыбы-иглы].

[Аккуратно отодвигая дары моря], [я доставал {с полок} книги], [садился по-турецки {в кресло} {с самшитовыми подлокотниками} и читал]. [Листал страницы {правой рукой}], [а левая сжимала кусок хлеба {с маслом} и сахаром]. [Откусывал задумчиво и читал], [и са

– [Ну нет], – [сказали финикийцы]. – [Мы люди работящие], [ремесленники и мореплаватели], [и нам {не нужна изощрённая каллиграфия}], [пусть {у нас} будет письменность попроще].

[И придумали буквы] – [так получился алфавит]. [Люди стали писать буквами], [и чем дальше], [тем быстрее]. [А чем быстрее они писали], [тем некрасивее {у них} получалось]. [Больше всех писали врачи]: [они выписывали рецепты]. [Поэтому {у некоторых} {из них} {до сих пор} {такой почерк}], [что пишут они вроде бы буквы], [а выходят иероглифы].

[{Древние греки} придумали Олимпийские игры], [пока вели одну {из своих нескончаемых войн}]. [{Основных причин} было две]: [во-первых], [{во время} баталий солдатам и офицерам некогда было заниматься спортом], [а ведь эллины] ([так называли себя {древние греки}]) [стремились тренироваться {всё время}], [{не занятое упражнениями} {в философии}]; [во-вторых], [воинам хотелось поскорее вернуться домой], [а отпуск {на войне} не предоставлялся]. [Было ясно], [что войска нуждалис

[Звуки же {собственной жизни} были столь скудны и {вопиюще незначительны}], [что Бах разучился их слышать]: [вычленял {в общем звуковом потоке} и пропускал {мимо ушей}]. [Дребезжало {под порывами} ветра стекло единственного {в комнате} окна], [потрескивал давно {не чищенный дымоход}], [изредка посвистывала откуда-то {из-под печи} {седая мышь}]. [Вот], [пожалуй], [и все]. [Слушать {большую жизнь} было {не в пример} интереснее]. [Иногда], [заслушавшись], [Бах даже забывал], [что он и {сам часть} этого мира], [что и он мог бы], [выйдя {на крыльцо}], [присоединиться {к многоголосью}]: [спеть что-нибудь задорное], [или громко хлопнуть дверью], [или], [{на худой конец}], [просто чихнуть]. [Но Бах предпочитал слушать].

[В шесть утра], [{тщательно одетый} и причесанный], [он уже стоял {у пришкольной колокольни} {с карманными часами} {в руках}]. [Дождавшись], [когда обе стрелки сольются {в единую линию}] ([часовая на шести], [минутная на двенадцати]), [что есть силы дергал {за веревку}] – [и {